# SSM Driver
So what are we trying to do here? We're basically seeing if we can convince an ANN to predict the components of the SSM needed to match the measurements given as input. That way we can take 1D measurements and convert them into semi-realistic 3D data which would be nice. How we exactly are going to manage that? I honestly don't know. I guess the things we need are:
1. **Dataloader** - a means of generating pairs of 3D models, their components and their measurements. That means I actually need to nail down how we define these measurements in the first place which isnt entirely obvious. I guess I just go with something that looks reasonable for now and we can refine exactly where the measurements are taken at a later date. Either way we're going to be giving it 4 circumfrential measures, 2 widths and a length and we'll see what comes out of it. 
2. **Loss function** - How exactly are we defining this loss - probably easiest by just using MSE between the 2 sets of measurements but it might be worth normalising them against the reference measure so that everything is of the same magnitude. Also probably worth creating a class or something that can just be passed the verts and output a set of measurements - this probably should be a `nn.Module` shouldn't it so it gets those nice properties? Idk maybe that doesnt matter too much?
3. **Model** - This is probably the simplest part of the whole shabang. We can just start with a really simple dense network and see what happens. Maybe throw in some normalisation but really this should be as simple as possible.


Something I havent really thought too much about is that I need to create these limbs based on my components 

In [17]:
import torch
from torch import nn
import subprocess
import os
import numpy as np
import igl
import wandb

from functools import partial
import measure_limbs

In [18]:
config = {
    "lr": 1e-3,
    "eta_min": 0.00001,
    "batch_size": 64,
    "log": False,
    "seed": 42,
}

torch.manual_seed(config["seed"])

if config["log"]:
    wandb.init(project="PointNet++ - FaceScanData", config=config)

In [19]:
class Measurements(nn.Module):
    def __init__(self, edge2vert, face2edge, details, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.edge2vert = edge2vert
        self.face2edge = face2edge
        self.details = details
        self.measures = []

        for detail in details:
            match detail["type"]:
                case "width":
                    measure = partial(measure_limbs.measure_width,
                        edge2vert=self.edge2vert,
                        plane_point=detail["plane_point"],
                        plane_normal=detail["plane_normal"],
                        plane_direction=detail["plane_direction"],
                    )
                case "length":
                    measure = partial(measure_limbs.measure_length,
                        v1=detail["v1"], v2=detail["v2"], direction=detail["direction"]
                    )
                case "circumference":
                    measure = partial(measure_limbs.measure_planar_circumference,
                        edge2vert=self.edge2vert,
                        face2edge=self.face2edge,
                        plane_point=detail["plane_point"],
                        plane_normal=detail["plane_normal"],
                    )
            self.measures.append(measure)

    def forward(self, x):
        return torch.tensor([measure(x) for measure in self.measures])


In [20]:
class LegMeasurementDataset(torch.utils.data.Dataset):
    def __init__(self, measure, batch_size=64, path="./stls", dtype=torch.float64):
        super().__init__()
        self.dtype = dtype
        self.measure = measure
        self.batch_size = batch_size
        self.path = path
        self.loaded_components = {}
        self.raw_components = torch.load("./data_components/vert_components.pt").to(dtype)
        self.mean_verts = torch.load("./data_components/mean_verts.pt").to(dtype)
        self.face2vert = torch.load("./data_components/face2vert.pt")
        self.vert_mapping = torch.load("./data_components/vert_mapping.pt")

        # Remove all .npy files from the specified path
        for file in os.listdir(self.path):
            if file.endswith(".npy"):
                os.remove(os.path.join(self.path, file))

        self.generate_data(0)
        self.generate_data(self.batch_size)

    def __len__(self):
        return 100_000

    def __getitem__(self, index):
        if index % self.batch_size == 0:
            self.generate_data(index + self.batch_size*2)
            ith_dataset = index // self.batch_size
            if ith_dataset >= 2:
                self.delete_data((ith_dataset - 2)*self.batch_size)

        components = self.loaded_components[(index // self.batch_size)*self.batch_size][
            index % self.batch_size
        ]

        verts = self.mean_verts + torch.sum(self.raw_components * components[:, None], dim=0).reshape_as(self.mean_verts)
        verts = verts[self.vert_mapping]

        measurements = self.measure.forward(verts)

        return measurements, components
        # return verts, self.face2vert, measurements, components
    

    def get_verts(self, components):
        verts = self.mean_verts + torch.sum(self.raw_components * components[:, None], dim=0).reshape_as(self.mean_verts)
        verts = verts[self.vert_mapping]

        return verts
    
    def get_measures(self, components=None, verts=None):
        if components is not None:
            verts = self.get_verts(components)
        
        return self.measure.forward(verts)

    def generate_data(self, start):
        cmd = [
            "./generate_limbs.sh",
            "--num_limbs",
            f"{self.batch_size}",
            "--path",
            self.path,
            "--start",
            f"{start}",
            "--save_mesh",
            "0",
        ]
        if os.name == "nt":  # Windows
            cmd = ["wsl", "-e"] + cmd

        subprocess.run(cmd, check=True)

        while True:
            try:
                components = np.load(f"{self.path}/components_{start:05d}.npy")
            except FileNotFoundError:
                continue
            else:
                self.loaded_components[start] = torch.tensor(components, dtype=self.dtype)
                break

    def delete_data(self, start):
        # for i in range(start, start + self.batch_size):
            # try:
            #     os.remove(f"{self.path}/limb_{i:05d}.npy")
            # except FileNotFoundError:
            #     pass

        os.remove(f"{self.path}/components_{start:05d}.npy")
        self.loaded_components.pop(start)

In [21]:
dtype = torch.float32

mean_verts = torch.load("./data_components/mean_verts.pt").to(dtype)
face2vert = torch.load("./data_components/face2vert.pt")

edge2vert, face2edge, edge2face = igl.edge_topology(
    mean_verts.numpy(), face2vert.numpy()
)

measurement_details = (
        {
            "type": "circumference",
            "plane_point": torch.tensor([[0, 0, 0]], dtype=dtype),
            "plane_normal": torch.tensor([[0, 0, 1]], dtype=dtype),
        },
    )
    
measure = Measurements(
    edge2vert,
    face2edge,
    measurement_details,
)

In [22]:
class Model(nn.Module):
    def __init__(self, input_dims, output_dims, activation=nn.ReLU(inplace=True)):
        super().__init__()

        self.layers = nn.ParameterList([
            nn.Linear(input_dims, 256),
            activation,
            nn.Linear(256, 1024),
            activation,
            nn.Linear(1024, 1024),
            activation,
            nn.Linear(1024, 128),
            activation,
            nn.Linear(128, output_dims)
        ])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)

        return x

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = LegMeasurementDataset(measure, batch_size=config["batch_size"], dtype=dtype)
dataloader = torch.utils.data.DataLoader(dataset, config["batch_size"], shuffle=False)
loss_func = nn.MSELoss().to(device)


model = Model(len(measurement_details), dataset.raw_components.shape[0]).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=config["lr"], weight_decay=1e-4)

In [24]:
for i, (measures, components) in enumerate(dataloader):
    measures = measures.to(device)
    components = components.to(device)
    
    preds = model(measures)

    loss = loss_func(preds, components)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if config["log"]:
        wandb.log({
            "loss": loss
        })
    else: 
        print(loss)

tensor(33.5256, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(33.1391, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(32.6755, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(31.9995, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(31.0987, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(30.0870, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(29.2366, device='cuda:0', grad_fn=<MseLossBackward0>)


Traceback (most recent call last):
  File "/opt/app/working/GenerateRandomLimbs.py", line 48, in <module>
    import matplotlib.pyplot as plt
  File "/usr/local/lib/python3.9/site-packages/matplotlib/pyplot.py", line 55, in <module>
    import matplotlib.colorbar
  File "/usr/local/lib/python3.9/site-packages/matplotlib/colorbar.py", line 19, in <module>
    from matplotlib import _api, cbook, collections, cm, colors, contour, ticker
  File "/usr/local/lib/python3.9/site-packages/matplotlib/contour.py", line 15, in <module>
    from matplotlib.backend_bases import MouseButton
  File "/usr/local/lib/python3.9/site-packages/matplotlib/backend_bases.py", line 49, in <module>
    from matplotlib import (
  File "/usr/local/lib/python3.9/site-packages/matplotlib/text.py", line 16, in <module>
    from .font_manager import FontProperties
  File "/usr/local/lib/python3.9/site-packages/matplotlib/font_manager.py", line 1588, in <module>
    fontManager = _load_fontmanager()
  File "/usr/local/

KeyboardInterrupt: 